In [360]:
import pandas as pd
from pulp import *
import requests
import json
from pprintpp import pprint as pp
from bs4 import BeautifulSoup as bs
pd.set_option("display.max_columns", None)

In [361]:
base_url = 'https://fantasy.premierleague.com/api/'

r = requests.get(base_url + 'bootstrap-static/')
data = r.json()

pp(data)

{
    'element_stats': [
        {'label': 'Minutes played', 'name': 'minutes'},
        {'label': 'Goals scored', 'name': 'goals_scored'},
        {'label': 'Assists', 'name': 'assists'},
        {'label': 'Clean sheets', 'name': 'clean_sheets'},
        {'label': 'Goals conceded', 'name': 'goals_conceded'},
        {'label': 'Own goals', 'name': 'own_goals'},
        {'label': 'Penalties saved', 'name': 'penalties_saved'},
        {'label': 'Penalties missed', 'name': 'penalties_missed'},
        {'label': 'Yellow cards', 'name': 'yellow_cards'},
        {'label': 'Red cards', 'name': 'red_cards'},
        {'label': 'Saves', 'name': 'saves'},
        {'label': 'Bonus', 'name': 'bonus'},
        {'label': 'Bonus Points System', 'name': 'bps'},
        {'label': 'Influence', 'name': 'influence'},
        {'label': 'Creativity', 'name': 'creativity'},
        {'label': 'Threat', 'name': 'threat'},
        {'label': 'ICT Index', 'name': 'ict_index'},
    ],
    'element_types': [
       

            'influence': '359.6',
            'influence_rank': 183,
            'influence_rank_type': 71,
            'minutes': 2027,
            'news': '',
            'news_added': None,
            'now_cost': 50,
            'own_goals': 0,
            'penalties_missed': 0,
            'penalties_order': None,
            'penalties_saved': 0,
            'penalties_text': '',
            'photo': '167199.jpg',
            'points_per_game': '3.0',
            'red_cards': 0,
            'saves': 0,
            'second_name': 'Partey',
            'selected_by_percent': '0.5',
            'special': False,
            'squad_number': None,
            'status': 'a',
            'team': 1,
            'team_code': 3,
            'threat': '394.0',
            'threat_rank': 99,
            'threat_rank_type': 59,
            'total_points': 71,
            'transfers_in': 0,
            'transfers_in_event': 0,
            'transfers_out': 0,
            'transfers_out_event': 

            'ict_index_rank_type': 3,
            'id': 13,
            'in_dreamteam': False,
            'influence': '790.4',
            'influence_rank': 23,
            'influence_rank_type': 9,
            'minutes': 2978,
            'news': '',
            'news_added': None,
            'now_cost': 80,
            'own_goals': 0,
            'penalties_missed': 0,
            'penalties_order': 1,
            'penalties_saved': 0,
            'penalties_text': '',
            'photo': '223340.jpg',
            'points_per_game': '4.7',
            'red_cards': 0,
            'saves': 0,
            'second_name': 'Saka',
            'selected_by_percent': '16.6',
            'special': False,
            'squad_number': None,
            'status': 'a',
            'team': 1,
            'team_code': 3,
            'threat': '1434.0',
            'threat_rank': 4,
            'threat_rank_type': 2,
            'total_points': 179,
            'transfers_in': 0,
            'tr

            'penalties_saved': 0,
            'penalties_text': '',
            'photo': '444145.jpg',
            'points_per_game': '3.8',
            'red_cards': 1,
            'saves': 0,
            'second_name': 'Martinelli Silva',
            'selected_by_percent': '12.9',
            'special': False,
            'squad_number': None,
            'status': 'a',
            'team': 1,
            'team_code': 3,
            'threat': '786.0',
            'threat_rank': 32,
            'threat_rank_type': 20,
            'total_points': 110,
            'transfers_in': 0,
            'transfers_in_event': 0,
            'transfers_out': 0,
            'transfers_out_event': 0,
            'value_form': '0.0',
            'value_season': '18.3',
            'web_name': 'Martinelli',
            'yellow_cards': 1,
        },
        {
            'assists': 0,
            'bonus': 0,
            'bps': 35,
            'chance_of_playing_next_round': None,
            'chance_of_p

            'ict_index_rank_type': 184,
            'id': 26,
            'in_dreamteam': False,
            'influence': '0.0',
            'influence_rank': 462,
            'influence_rank_type': 184,
            'minutes': 0,
            'news': '',
            'news_added': None,
            'now_cost': 45,
            'own_goals': 0,
            'penalties_missed': 0,
            'penalties_order': None,
            'penalties_saved': 0,
            'penalties_text': '',
            'photo': '462424.jpg',
            'points_per_game': '0.0',
            'red_cards': 0,
            'saves': 0,
            'second_name': 'Saliba',
            'selected_by_percent': '0.7',
            'special': False,
            'squad_number': None,
            'status': 'a',
            'team': 1,
            'team_code': 3,
            'threat': '0.0',
            'threat_rank': 450,
            'threat_rank_type': 184,
            'total_points': 0,
            'transfers_in': 0,
            

...

            'highest_scoring_entry': None,
            'id': 9,
            'is_current': False,
            'is_next': False,
            'is_previous': False,
            'most_captained': None,
            'most_selected': None,
            'most_transferred_in': None,
            'most_vice_captained': None,
            'name': 'Gameweek 9',
            'top_element': None,
            'top_element_info': None,
            'transfers_made': 0,
        },
        {
            'average_entry_score': 0,
            'chip_plays': [
                {'chip_name': 'bboost', 'num_played': 50931},
                {'chip_name': 'freehit', 'num_played': 96697},
                {'chip_name': 'wildcard', 'num_played': 189850},
                {'chip_name': '3xc', 'num_played': 156449},
            ],
            'cup_leagues_created': False,
            'data_checked': False,
            'deadline_time': '2022-10-08T12:30:00Z',
            'deadline_time_epoch': 1665232200,
            'deadli

            'name': 'Gameweek 18',
            'top_element': None,
            'top_element_info': None,
            'transfers_made': 0,
        },
        {
            'average_entry_score': 0,
            'chip_plays': [
                {'chip_name': 'bboost', 'num_played': 23155},
                {'chip_name': 'freehit', 'num_played': 704625},
                {'chip_name': 'wildcard', 'num_played': 89658},
                {'chip_name': '3xc', 'num_played': 45125},
            ],
            'cup_leagues_created': False,
            'data_checked': False,
            'deadline_time': '2023-01-02T13:30:00Z',
            'deadline_time_epoch': 1672666200,
            'deadline_time_game_offset': 0,
            'finished': False,
            'h2h_ko_matches_created': False,
            'highest_score': None,
            'highest_scoring_entry': None,
            'id': 19,
            'is_current': False,
            'is_next': False,
            'is_previous': False,
            'mos

                {'chip_name': 'freehit', 'num_played': 199886},
                {'chip_name': 'wildcard', 'num_played': 300722},
                {'chip_name': '3xc', 'num_played': 51262},
            ],
            'cup_leagues_created': False,
            'data_checked': False,
            'deadline_time': '2023-03-18T13:30:00Z',
            'deadline_time_epoch': 1679146200,
            'deadline_time_game_offset': 0,
            'finished': False,
            'h2h_ko_matches_created': False,
            'highest_score': None,
            'highest_scoring_entry': None,
            'id': 28,
            'is_current': False,
            'is_next': False,
            'is_previous': False,
            'most_captained': 233,
            'most_selected': 233,
            'most_transferred_in': 127,
            'most_vice_captained': 233,
            'name': 'Gameweek 28',
            'top_element': None,
            'top_element_info': None,
            'transfers_made': 12705647,
        

            'most_selected': None,
            'most_transferred_in': None,
            'most_vice_captained': None,
            'name': 'Gameweek 36',
            'top_element': None,
            'top_element_info': None,
            'transfers_made': 0,
        },
        {
            'average_entry_score': 0,
            'chip_plays': [],
            'cup_leagues_created': False,
            'data_checked': False,
            'deadline_time': '2023-05-20T12:30:00Z',
            'deadline_time_epoch': 1684585800,
            'deadline_time_game_offset': 0,
            'finished': False,
            'h2h_ko_matches_created': False,
            'highest_score': None,
            'highest_scoring_entry': None,
            'id': 37,
            'is_current': False,
            'is_next': False,
            'is_previous': False,
            'most_captained': None,
            'most_selected': None,
            'most_transferred_in': None,
            'most_vice_captained': None,
        

            'strength_defence_away': 1160,
            'strength_defence_home': 1080,
            'strength_overall_away': 1100,
            'strength_overall_home': 1090,
            'team_division': None,
            'unavailable': False,
            'win': 0,
        },
        {
            'code': 11,
            'draw': 0,
            'form': None,
            'id': 8,
            'loss': 0,
            'name': 'Everton',
            'played': 0,
            'points': 0,
            'position': 0,
            'pulse_id': 7,
            'short_name': 'EVE',
            'strength': 3,
            'strength_attack_away': 1090,
            'strength_attack_home': 1130,
            'strength_defence_away': 1110,
            'strength_defence_home': 1110,
            'strength_overall_away': 1100,
            'strength_overall_home': 1050,
            'team_division': None,
            'unavailable': False,
            'win': 0,
        },
        {
            'code': 54,
            

In [ ]:
'''
https://medium.com/@frenzelts/fantasy-premier-league-api-endpoints-a-detailed-guide-acbd5598eb19

General:
Endpoint path: bootstrap-static/
-------------------------------
    events: Basic information of every Gameweek such as average score, highest score, top scoring player, most captained, etc.
    game_settings: The game settings and rules. (not important)
    phases: Phases of FPL season. (not really important)
    teams: Basic information of current Premier League clubs.
    total_players: Total FPL players.
    elements: Information of all Premier League players including points, status, value, match stats (goals, assists, etc.), ICT index, etc.
    element_types: Basic information about player’s position (GK, DEF, MID, FWD).

Fixtures:
Endpoint path: fixtures/
------------------------
    Now here’s the explanation of some of the JSON elements:
    event refers to the event id in events section of the bootstrap-static data.
    team_a and team_h refers to the team id in teams section of the bootstrap-static data. team_a for the away team and team_h for the home team.
    team_h_difficulty and team_a_difficulty is the FDR value calculated by FPL.
    stats contains a list of match facts that affect points of a player. 
    It consists of goals_scored, assists, own_goals, penalties_saved, penalties_missed, yellow_cards, red_cards, saves, bonus, and bps data.
    value is the amount and element refers to the element id in elements section of the bootstrap-static data. 
    For example, in image below we can see goals_scored data from the away team. These information can be read as: Aubameyang, Lacazette, and Gabriel each scored one goal for the away team (which is Arsenal).

Gameweek live data:
Endpoint path: event/{event_id}/live/
---------------------------
This endpoint returns a list of players’ information in that specific Gameweek. 
Each element is divided into three section:
    id: Refers to the element id in elements section of the bootstrap-static data.
    
    stats: Player’s match stats including goals, assists, etc.
    stats section of an element
    stats section of an element
    
    explain: Breakdown of a player’s event points. For example, let’s see from image below 
    (Son Heung-Min’s data in Gameweek 4 match against Manchester United, 2020/21). 
    From image below we can see that Son played 72 minutes, so 2 points given. 
    We can also see that he scored 2 goals which resulted in 10 points, one assist which resulted in 3 points, and 3 bonus points. 
    Overall, he scored a solid 18 points in Gameweek 4. 
    (Sorry for the example United fans and anyone who didn’t start Son that week)

Player’s Detailed Data:
Endpoint path: element-summary/{element_id}/
--------------------------------------------
This endpoint returns a player’s detailed information divided into 3 section:
    fixtures: A list of player’s remaining fixtures of the season. 
    Each fixture object consists of these information below:

    history: A list of player’s previous fixtures and its match stats.

    history_past: A list of player’s previous seasons and its seasonal stats.
'''

In [400]:
class FPL:
    base_url = 'https://fantasy.premierleague.com/api/'
    def __init__(self, endpoint_name, endpoint):
        self.endpoint_name = endpoint_name
        self.endpoint      = endpoint
        
    def getEndpointData(self, ep=''):
        # Done-- call any endpoint and pp the data
        combine_url = FPL.base_url + ep + "/"
        r    = requests.get(combine_url)
        data = r.json()
        return data
        
    def displayEndpoint(self):
        d = f'{self.endpoint_name} is at this endpoint: {self.endpoint}'
        pp(d)
        
fpl_bootstrap = FPL('bootstrap', 'bootstrap-static/')

fpl_fixtures = FPL('fixtures', 'fixtures/')

fpl_players = FPL('player_stats', 'element-summary/')

fpl_gameweek = FPL('gameweek', 'event/')

fpl_myteam = FPL('my_team', 'my-team/2751708/')

fpl_fixtures.displayEndpoint()


'fixtures is at this endpoint: fixtures/'


In [256]:
class Player(FPL):
    def __init__(self, endpoint_name, endpoint, player_name, element_id):
        super().__init__(endpoint_name, endpoint)
        self.element_id  = element_id
        self.player_name = player_name 
    
    def getPlayerInfo(self):
        # Done -- get a player's detailed stats -- Sections include: fixtures, history, history_past
        
        player_endpoint = self.endpoint + self.element_id
        r    = FPL.getEndpointData(self, ep=player_endpoint)
        pp(r)
        
    def getPlayerUpcomingFixtures(self):
        # Done -- use element_id in endpoint and output their upcoming fixtures for this season
        player_endpoint = self.endpoint + self.element_id
        r    = FPL.getEndpointData(self, ep=player_endpoint)
        df   = pd.DataFrame(r['fixtures'])
        return df
        
    def getPlayerPastFixtures(self):
        # Done -- input player id as query string and output their past fixtures for this season
        player_endpoint = self.endpoint + self.element_id
        r    = FPL.getEndpointData(self, ep=player_endpoint)
        df   = pd.DataFrame(r['history'])
        return df
    
    def getPlayerHistoricalFixtures(self):
        # Done -- input player id as query string and output their seasonal stats for previous seasons
        player_endpoint = self.endpoint + self.element_id
        r    = FPL.getEndpointData(self, ep=player_endpoint)
        df   = pd.DataFrame(r['history_past'])
        return df
    
fpl_players = Player('player_stats', 'element-summary/', 'Mo Salah', '4')

#fpl_players.getPlayerHistoricalFixtures()

fpl_players.getPlayerInfo()

{
    'fixtures': [
        {
            'code': 2292810,
            'difficulty': 2,
            'event': 1,
            'event_name': 'Gameweek 1',
            'finished': False,
            'id': 1,
            'is_home': False,
            'kickoff_time': '2022-08-05T19:00:00Z',
            'minutes': 0,
            'provisional_start_time': False,
            'team_a': 1,
            'team_a_score': None,
            'team_h': 7,
            'team_h_score': None,
        },
        {
            'code': 2292820,
            'difficulty': 3,
            'event': 2,
            'event_name': 'Gameweek 2',
            'finished': False,
            'id': 11,
            'is_home': True,
            'kickoff_time': '2022-08-13T14:00:00Z',
            'minutes': 0,
            'provisional_start_time': False,
            'team_a': 10,
            'team_a_score': None,
            'team_h': 1,
            'team_h_score': None,
        },
        {
            'code': 2292830,
        

            'team_a_score': None,
            'team_h': 5,
            'team_h_score': None,
        },
        {
            'code': 2292990,
            'difficulty': 3,
            'event': 19,
            'event_name': 'Gameweek 19',
            'finished': False,
            'id': 181,
            'is_home': True,
            'kickoff_time': '2023-01-02T15:00:00Z',
            'minutes': 0,
            'provisional_start_time': False,
            'team_a': 15,
            'team_a_score': None,
            'team_h': 1,
            'team_h_score': None,
        },
        {
            'code': 2293008,
            'difficulty': 4,
            'event': 20,
            'event_name': 'Gameweek 20',
            'finished': False,
            'id': 199,
            'is_home': False,
            'kickoff_time': '2023-01-14T15:00:00Z',
            'minutes': 0,
            'provisional_start_time': False,
            'team_a': 1,
            'team_a_score': None,
            'team_h': 18,


In [363]:
class Fixture(FPL):
    def __init__(self, endpoint_name, endpoint):
        super().__init__(endpoint_name, endpoint)
        
    def getFixtureData(self):
        # Done-- call fixtures endpoint and input data into dataframe
        r    = requests.get(FPL.base_url + self.endpoint)
        data = r.json()
        df   = pd.DataFrame(data)
        return df
            
fpl_fixtures = Fixture('fixtures', 'fixtures/')

fpl_fixtures.getFixtureData()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2292810,1,False,False,1,2022-08-05T19:00:00Z,0,False,False,1,None,7,None,[],3,2,74911
1,2292813,1,False,False,4,2022-08-06T11:30:00Z,0,False,False,12,None,9,None,[],5,2,74914
2,2292811,1,False,False,2,2022-08-06T14:00:00Z,0,False,False,2,None,3,None,[],2,2,74912
3,2292814,1,False,False,5,2022-08-06T14:00:00Z,0,False,False,20,None,11,None,[],2,2,74915
4,2292816,1,False,False,7,2022-08-06T14:00:00Z,0,False,False,16,None,15,None,[],2,3,74917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2293185,38,False,False,376,2023-05-28T15:00:00Z,0,False,False,3,None,8,None,[],2,2,75286
376,2293186,38,False,False,377,2023-05-28T15:00:00Z,0,False,False,18,None,11,None,[],3,2,75287
377,2293187,38,False,False,378,2023-05-28T15:00:00Z,0,False,False,19,None,10,None,[],3,4,75288
378,2293188,38,False,False,379,2023-05-28T15:00:00Z,0,False,False,9,None,14,None,[],2,4,75289


In [386]:
class MyTeam(FPL):
    def __init__(self, endpoint_name, endpoint):
        super().__init__(endpoint_name, endpoint)
    
    def getMyTeamInfo(self):
        # Done -- post login details to fpl, get my team selection in dataframe
        team_id = 2751708
        current_week = "0"
        url_team = f"https://fantasy.premierleague.com/api/my-team/{team_id}/"
        session = requests.session()
        url = 'https://users.premierleague.com/accounts/login/'
        payload = {
         'login': 'slhillmn1@gmail.com',
         'password': '@TimSykes1995',
         'redirect_uri': 'https://fantasy.premierleague.com/a/login',
         'app': 'plfpl-web'
        }

        headers = {
           'authority': 'users.premierleague.com' ,
           'cache-control': 'max-age=0' ,
           'upgrade-insecure-requests': '1' ,
           'origin': 'https://fantasy.premierleague.com' ,
           'content-type': 'application/x-www-form-urlencoded' ,
           'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36' ,
           'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' ,
           'sec-fetch-site': 'same-site' ,
           'sec-fetch-mode': 'navigate' ,
           'sec-fetch-user': '?1' ,
           'sec-fetch-dest': 'document' ,
           'referer': 'https://fantasy.premierleague.com/my-team' ,
           'accept-language': 'en-US,en;q=0.9,he;q=0.8' ,
        }

        session.post(url, data=payload, headers=headers)
        try:
            response = session.get(url_team)
            data     = json.loads(response.content)
            df       = pd.DataFrame(data['picks'])
            return df
            
        except requests.exceptions.RequestException:
            print('HTTP Request failed')
        
            
    def joinMyTeamAndEnrich(self):
        # Done -- enrich my team data call with data about players and teams
        # read in player stats csv
        data = pd.read_csv('/Users/stephenhill/Documents/Code Development/Data Science/fpl_team_project/fpl_notebooks/fpl_data_store/player_stats.csv')
        players_df = pd.DataFrame(data)
        
        # read in player positions csv
        data = pd.read_csv('/Users/stephenhill/Documents/Code Development/Data Science/fpl_team_project/fpl_notebooks/fpl_data_store/player_positions.csv')
        pos_df   = pd.DataFrame(data)
        
        # read in teams csv
        data = pd.read_csv('/Users/stephenhill/Documents/Code Development/Data Science/fpl_team_project/fpl_notebooks/fpl_data_store/teams.csv')
        teams_df   = pd.DataFrame(data)
        
        # getMyTeamInfo dataframe and join to df on element and id
        my_team_df = fpl_myteam.getMyTeamInfo()
        
        # merge dataframes
        players_pos_join       = pd.merge(players_df, pos_df, left_on='element_type', right_on='id')
        players_pos_teams_join = pd.merge(players_pos_join, teams_df, left_on='team', right_on='id')
        players_pos_teams_myteam_join = pd.merge(my_team_df, players_pos_teams_join, left_on='element', right_on='id_x')


        df = players_pos_teams_myteam_join[['id_x', 'first_name', 'second_name', 'name', 'plural_name_short','is_captain','is_vice_captain']]
        df = df.sort_values(by='name')
        #df.to_csv('/Users/stephenhill/Documents/Code Development/Data Science/fpl_team_project/fpl_notebooks/fpl_data_store/my_team_gw1.csv')

        return df
    
    def getMyTeamIds(self):
        df = self.joinMyTeamAndEnrich()
        df = df[['id_x', 'first_name', 'second_name']] 
        df_list = df.values.tolist()
        return df_list
    
    def getMyTeamNextFixtureDifficulty(self):
        # Done -- get player's next game difficulty to gauge who should be captain or vice
        id_name_list = self.getMyTeamIds()
        
        team_difficulty_list = []
        
        for i in id_name_list:
            print(i[1], i[2])
            player_id = f'{i[0]}'
            player_endpoint = 'element-summary/' + player_id
            r    = FPL.getEndpointData(self, ep=player_endpoint)
            fixture_week = r['fixtures'][0]
            fixture_week['first_name'] = i[1]
            fixture_week['second_name'] = i[2]
            
            team_difficulty_list.append(fixture_week)
        
        df = pd.DataFrame(team_difficulty_list)
        df = df.sort_values(by='difficulty')
        #df.to_csv('/Users/stephenhill/Documents/Code Development/Data Science/fpl_team_project/fpl_notebooks/fpl_data_store/my_team_gw1_difficulty.csv')
        
        return df
    
            
            
fpl_myteam = MyTeam('my_team', 'my-team/2751708/')

#fpl_myteam.getMyTeamInfo()

#fpl_myteam.getMyTeamIds()

fpl_myteam.joinMyTeamAndEnrich()

#fpl_myteam.getMyTeamNextFixtureDifficulty()


,id_x,first_name,second_name,name,plural_name_short,is_captain,is_vice_captain
5,13,Bukayo,Saka,Arsenal,MID,False,False
10,28,Gabriel,Fernando de Jesus,Arsenal,FWD,True,False
12,16,Gabriel,dos Santos Magalhães,Arsenal,DEF,False,False
3,43,Matty,Cash,Aston Villa,DEF,False,False
9,80,Ivan,Toney,Brentford,FWD,False,False
6,130,Jorge Luiz,Frello Filho,Chelsea,MID,False,False
0,281,Alisson,Ramses Becker,Liverpool,GKP,False,False
1,280,Virgil,van Dijk,Liverpool,DEF,False,False
2,285,Trent,Alexander-Arnold,Liverpool,DEF,False,False
4,306,João,Cancelo,Man City,DEF,False,False


In [370]:
class Captains(Player):
    # trying to get player data and next game difficulty to guide captaincy decision
    # gameweek
    # which player
    event_num = 1
    def __init__(self, endpoint_name, endpoint, player_name, element_id):
        super().__init__(endpoint_name, endpoint, player_name, element_id)
        
    def getPlayerFixture(self):
        player_fixtures = self.getPlayerUpcomingFixtures()
        return player_fixtures

fpl_players = Captains('player_stats', 'element-summary/', 'Mo Salah', '4')
 
fpl_players.getPlayerFixture()

,id,code,team_h,team_h_score,team_a,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty
0,1,2292810,7,None,1,None,1,False,0,False,2022-08-05T19:00:00Z,Gameweek 1,False,2
1,11,2292820,1,None,10,None,2,False,0,False,2022-08-13T14:00:00Z,Gameweek 2,True,3
2,21,2292830,3,None,1,None,3,False,0,False,2022-08-20T16:30:00Z,Gameweek 3,False,2
3,31,2292840,1,None,9,None,4,False,0,False,2022-08-27T16:30:00Z,Gameweek 4,True,2
4,41,2292851,1,None,2,None,5,False,0,False,2022-08-31T18:30:00Z,Gameweek 5,True,2
5,56,2292865,14,None,1,None,6,False,0,False,2022-09-04T15:30:00Z,Gameweek 6,False,4
6,61,2292871,1,None,8,None,7,False,0,False,2022-09-11T13:00:00Z,Gameweek 7,True,2
7,72,2292881,4,None,1,None,8,False,0,False,2022-09-18T13:00:00Z,Gameweek 8,False,2
8,81,2292891,1,None,18,None,9,False,0,False,2022-10-01T14:00:00Z,Gameweek 9,True,3
9,91,2292901,1,None,12,None,10,False,0,False,2022-10-08T14:00:00Z,Gameweek 10,True,5


In [401]:
class GameweekLive(FPL):
    # Endpoint path: event/4/live/
    def __init__(self, endpoint_name, endpoint):
        super().__init__(endpoint_name, endpoint)
    
    def getGameweekData(self):
        # for each player in team, we want their stats and explanation in a df
        # can only test when the gameweek has occurred
        print('input Gameweek:')
        gw = '1'
        gw_endpoint = self.endpoint
        gw_endpoint = gw_endpoint.format(gw=gw)
        print(gw_endpoint)
        gw_data = self.getEndpointData(ep=gw_endpoint)
        return gw_data
        
gw_1 = GameweekLive('gameweek_live', 'event/{gw}/live')

gw_1.getGameweekData()

input Gameweek:
event/1/live


{'elements': []}